In [1]:
import plotly.express as px
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
from IPython.display import display, HTML
import plotly.graph_objects as go
import json
import os
import time

In [2]:
logs_path = Path("logs")

# Standalone

In [3]:
logs_standalone = logs_path / "standalone.logs.json"

In [4]:
logs_standalone_df = pd.read_json(logs_standalone)
logs_standalone_df

,epoch,step,absolut_step,loss_d,loss_g,time_elapsed,start_time,end_time,fid_score,inception_score
0,0,0,0,1.507229,44.310856,0.788273,2024-05-13 12:10:17.801522970,2024-05-13 12:10:18.589796066,462.025726,1.0
1,1,0,16277,0.998532,56.282173,0.142990,2024-05-13 12:10:23.582053185,2024-05-13 12:10:23.725043058,NaN,NaN
2,2,0,32554,3.144789,44.293766,0.046519,2024-05-13 12:10:23.800288916,2024-05-13 12:10:23.846807957,NaN,NaN
3,3,0,48831,0.000039,37.916954,0.048004,2024-05-13 12:10:23.919428110,2024-05-13 12:10:23.967432022,NaN,NaN
4,4,0,65108,0.348700,11.308703,0.049462,2024-05-13 12:10:24.038013935,2024-05-13 12:10:24.087476015,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
385,385,0,6266645,0.704110,2.857518,0.066551,2024-05-13 12:14:25.269562960,2024-05-13 12:14:25.336113930,NaN,NaN
386,386,0,6282922,0.818662,2.074063,0.067292,2024-05-13 12:14:25.438248873,2024-05-13 12:14:25.505541086,NaN,NaN
387,387,0,6299199,1.253154,4.310959,0.051895,2024-05-13 12:14:25.599493027,2024-05-13 12:14:25.651387930,NaN,NaN
388,388,0,6315476,1.953032,3.215519,0.050762,2024-05-13 12:14:25.741305113,2024-05-13 12:14:25.792067051,NaN,NaN


In [5]:
px.line(logs_standalone_df[["absolut_step", "loss_g", "loss_d"]], x="absolut_step", y=["loss_g", "loss_d"], title="Losses")

In [6]:
px.line(logs_standalone_df[["absolut_step", "fid_score"]].dropna(), x="absolut_step", y="fid_score", title="Scores")

In [7]:
px.line(logs_standalone_df[["absolut_step", "inception_score"]].dropna(), x="absolut_step", y="inception_score", title="Scores")

# Distributed

In [8]:
logs_files = glob(str(logs_path / "*.logs.json"))
logs_files.pop(logs_files.index(str(logs_standalone)))
logs_files.sort()
logs_files

['logs/server_0.logs.json',
 'logs/worker_1.logs.json',
 'logs/worker_2.logs.json',
 'logs/worker_3.logs.json',
 'logs/worker_4.logs.json']

In [9]:
server_logs = logs_files.pop(0)
server_logs, logs_files

('logs/server_0.logs.json',
 ['logs/worker_1.logs.json',
  'logs/worker_2.logs.json',
  'logs/worker_3.logs.json',
  'logs/worker_4.logs.json'])

# Server

In [10]:
server_df = pd.read_json(server_logs, orient="index")
server_df["fid_score"] = server_df["fid_score"].apply(lambda x: 0 if x is None else x)
server_df["inception_score"] = server_df["inception_score"].apply(lambda x: 0 if x is None else x)
server_df["name"] = "server"
server_with_score_df = server_df[server_df["fid_score"].notnull() & server_df["inception_score"].notnull()]
display(server_df)
display(server_with_score_df)

,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-13 13:19:15.547255039,2.112973,1.715606e+09,2024-05-13 13:19:14.621806145,2024-05-13 13:19:13.685881138,0,2024-05-13 13:19:13.434282064,1.715606e+09,1.0,1.715606e+09,1.715606e+09,504.196655,1.715606e+09,server
1,2024-05-13 13:19:20.028450966,0.979838,1.715606e+09,2024-05-13 13:19:19.564069033,2024-05-13 13:19:19.153178215,1,2024-05-13 13:19:19.048613071,NaN,NaN,NaN,NaN,NaN,NaN,server
2,2024-05-13 13:19:20.922868013,0.893975,1.715606e+09,2024-05-13 13:19:20.439091921,2024-05-13 13:19:20.106369972,2,2024-05-13 13:19:20.028892994,NaN,NaN,NaN,NaN,NaN,NaN,server
3,2024-05-13 13:19:21.967429161,1.044072,1.715606e+09,2024-05-13 13:19:21.337628841,2024-05-13 13:19:20.999706984,3,2024-05-13 13:19:20.923357010,NaN,NaN,NaN,NaN,NaN,NaN,server
4,2024-05-13 13:19:22.799109221,0.831188,1.715606e+09,2024-05-13 13:19:22.390048027,2024-05-13 13:19:22.056450129,4,2024-05-13 13:19:21.967920780,NaN,NaN,NaN,NaN,NaN,NaN,server
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,2024-05-13 13:40:21.422496080,0.873828,1.715608e+09,2024-05-13 13:40:20.974724054,2024-05-13 13:40:20.629858017,1000,2024-05-13 13:40:20.548667908,1.715608e+09,1.0,1.715608e+09,1.715608e+09,422.319489,1.715608e+09,server
1001,2024-05-13 13:40:26.315452814,1.030728,1.715608e+09,2024-05-13 13:40:25.793985128,2024-05-13 13:40:25.393855095,1001,2024-05-13 13:40:25.284724951,NaN,NaN,NaN,NaN,NaN,NaN,server
1002,2024-05-13 13:40:27.170197964,0.843092,1.715608e+09,2024-05-13 13:40:26.747940063,2024-05-13 13:40:26.408451080,1002,2024-05-13 13:40:26.327105999,NaN,NaN,NaN,NaN,NaN,NaN,server
1003,2024-05-13 13:40:28.026266813,0.845277,1.715608e+09,2024-05-13 13:40:27.605067015,2024-05-13 13:40:27.257714987,1003,2024-05-13 13:40:27.180989981,NaN,NaN,NaN,NaN,NaN,NaN,server


,end_time,elapsed_time,logging_time_end,received_time,sent_time,epoch,start_time,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end,name
0,2024-05-13 13:19:15.547255039,2.112973,1.715606e+09,2024-05-13 13:19:14.621806145,2024-05-13 13:19:13.685881138,0,2024-05-13 13:19:13.434282064,1.715606e+09,1.0,1.715606e+09,1.715606e+09,504.196655,1.715606e+09,server
10,2024-05-13 13:19:27.759531021,0.822972,1.715606e+09,2024-05-13 13:19:27.341400146,2024-05-13 13:19:27.009082079,10,2024-05-13 13:19:26.936559200,1.715606e+09,1.0,1.715606e+09,1.715606e+09,448.988220,1.715606e+09,server
20,2024-05-13 13:19:40.044765949,0.823288,1.715606e+09,2024-05-13 13:19:39.622214794,2024-05-13 13:19:39.288590193,20,2024-05-13 13:19:39.221477985,1.715606e+09,1.0,1.715606e+09,1.715606e+09,458.858429,1.715606e+09,server
30,2024-05-13 13:19:52.567847967,0.864945,1.715606e+09,2024-05-13 13:19:52.109349012,2024-05-13 13:19:51.773888111,30,2024-05-13 13:19:51.702903032,1.715606e+09,1.0,1.715606e+09,1.715606e+09,396.578796,1.715606e+09,server
40,2024-05-13 13:20:06.007726192,0.859000,1.715606e+09,2024-05-13 13:20:05.562409878,2024-05-13 13:20:05.227952957,40,2024-05-13 13:20:05.148725986,1.715606e+09,1.0,1.715606e+09,1.715606e+09,429.240326,1.715606e+09,server
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,2024-05-13 13:39:29.139824867,0.877570,1.715608e+09,2024-05-13 13:39:28.677328825,2024-05-13 13:39:28.343269825,960,2024-05-13 13:39:28.262255192,1.715608e+09,1.0,1.715608e+09,1.715608e+09,394.533905,1.715608e+09,server
970,2024-05-13 13:39:42.350256205,0.872569,1.715608e+09,2024-05-13 13:39:41.917474985,2024-05-13 13:39:41.575385094,970,2024-05-13 13:39:41.477687120,1.715608e+09,1.0,1.715608e+09,1.715608e+09,412.536926,1.715608e+09,server
980,2024-05-13 13:39:55.371173859,0.812301,1.715608e+09,2024-05-13 13:39:54.977726936,2024-05-13 13:39:54.631934881,980,2024-05-13 13:39:54.558873177,1.715608e+09,1.0,1.715608e+09,1.715608e+09,409.928802,1.715608e+09,server
990,2024-05-13 13:40:08.354535818,0.934624,1.715608e+09,2024-05-13 13:40:07.835666895,2024-05-13 13:40:07.495035887,990,2024-05-13 13:40:07.419911861,1.715608e+09,1.0,1.715608e+09,1.715608e+09,496.233551,1.715608e+09,server


In [11]:
figure = go.Figure()
normalized_fid = server_with_score_df.copy()
lineplot_fid = px.line(
    normalized_fid,
    x="epoch",
    y="fid_score",
    title="Server logs",
).show()
lineplot_is = px.line(
    normalized_fid,
    x="epoch",
    y="inception_score",
    title="Server logs",
).show()
duration_plot = px.bar(
    normalized_fid,
    x="epoch",
    y="elapsed_time",
    title="Server logs",
    text="elapsed_time",
    template="plotly_white"
).show()

In [12]:
px.timeline(
    server_df,
    x_start="start_time",
    x_end="end_time",
    title="Server logs",
    template="plotly_white"
)

# Clients

In [13]:
def read_client_json(file_path: str):
    flattened_local_epochs = []
    flattened_swap = []
    flattened_send = []
    for outer_key, inner_dict in json.loads(open(file_path, "r").read()).items():
        root_attributes = {k: v for k, v in inner_dict.items() if not isinstance(v, dict)}
        if "start_time_swap" in root_attributes:
            flattened_swap.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_swap"],
                "end_time": root_attributes["end_time_swap"],
            })
        if "start_time_send" in root_attributes:
            flattened_send.append({
                "outer_epoch": outer_key,
                "start_time": root_attributes["start_time_send"],
                "end_time": root_attributes["end_time_send"],
            })
        for inner_key, values_dict in inner_dict.items():
            if not isinstance(values_dict, dict):
                continue
            flattened_local_epochs.append({
                "outer_epoch": outer_key,
                "local_epoch": inner_key,
                **values_dict,
                **root_attributes
            })

    # Convert flattened data to DataFrame
    name = os.path.basename(file_path).split(".")[0]
    
    df_local_epochs = pd.DataFrame(flattened_local_epochs)
    df_local_epochs["name"] = name

    df_swap = pd.DataFrame(flattened_swap)
    df_swap["name"] = f"{name}_swap"

    df_send = pd.DataFrame(flattened_send)
    df_send["name"] = f"{name}_send"

    return df_local_epochs, df_swap, df_send

In [14]:
clients_df = pd.concat([pd.concat(read_client_json(file)) for file in logs_files])

clients_df["start_time"] = pd.to_datetime(clients_df["start_time"], unit="s")
clients_df["end_time"] = pd.to_datetime(clients_df["end_time"], unit="s")

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_recv"
server_to_client_df["start_time"] = server_to_client_df["received_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

server_to_client_df = server_df.copy()
server_to_client_df["name"] = "server_sent"
server_to_client_df["end_time"] = server_to_client_df["sent_time"]
clients_df = pd.concat([clients_df, server_to_client_df])

clients_df

,outer_epoch,local_epoch,d_loss_real,d_loss_fake,d_total_loss,elapsed_time,start_time,end_time,start_time_send,end_time_send,...,logging_time_end,received_time,sent_time,epoch,is_calculation_time_start,inception_score,is_calculation_time_end,fid_calculation_time_start,fid_score,fid_calculation_time_end
0,0,0,0.480324,0.939980,1.420304,0.759346,2024-05-13 13:19:13.684431791,2024-05-13 13:19:14.441240072,1.715606e+09,1.715606e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0.419534,0.693147,1.112681,0.250546,2024-05-13 13:19:19.155570984,2024-05-13 13:19:19.399858952,1.715606e+09,1.715606e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0.417877,0.693147,1.111024,0.221346,2024-05-13 13:19:20.105675936,2024-05-13 13:19:20.320869923,1.715606e+09,1.715606e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,0.313711,1.313262,1.626973,0.214386,2024-05-13 13:19:21.000042915,2024-05-13 13:19:21.210474968,1.715606e+09,1.715606e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,0.346819,1.313262,1.660080,0.199863,2024-05-13 13:19:22.045061827,2024-05-13 13:19:22.237295151,1.715606e+09,1.715606e+09,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,NaN,NaN,NaN,NaN,NaN,0.873828,2024-05-13 13:40:20.548667908,2024-05-13 13:40:20.629858017,NaN,NaN,...,1.715608e+09,2024-05-13 13:40:20.974724054,2024-05-13 13:40:20.629858017,1000.0,1.715608e+09,1.0,1.715608e+09,1.715608e+09,422.319489,1.715608e+09
1001,NaN,NaN,NaN,NaN,NaN,1.030728,2024-05-13 13:40:25.284724951,2024-05-13 13:40:25.393855095,NaN,NaN,...,1.715608e+09,2024-05-13 13:40:25.793985128,2024-05-13 13:40:25.393855095,1001.0,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,0.843092,2024-05-13 13:40:26.327105999,2024-05-13 13:40:26.408451080,NaN,NaN,...,1.715608e+09,2024-05-13 13:40:26.747940063,2024-05-13 13:40:26.408451080,1002.0,NaN,NaN,NaN,NaN,NaN,NaN
1003,NaN,NaN,NaN,NaN,NaN,0.845277,2024-05-13 13:40:27.180989981,2024-05-13 13:40:27.257714987,NaN,NaN,...,1.715608e+09,2024-05-13 13:40:27.605067015,2024-05-13 13:40:27.257714987,1003.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
only_workers = [f"worker_{i+1}" for i in range(len(logs_files))]
only_workers

['worker_1', 'worker_2', 'worker_3', 'worker_4']

In [16]:
px.line(
    clients_df[clients_df["name"].isin(only_workers)],
    x="start_time",
    y="d_total_loss",
    color="name",
    title="Client logs",
    template="plotly_white"
)

In [17]:
clients_and_server_df = pd.concat(
    [
        clients_df[["start_time", "end_time", "name"]],
        server_df[["start_time", "end_time", "name"]],
    ]
)
clients_and_server_df

,start_time,end_time,name
0,2024-05-13 13:19:13.684431791,2024-05-13 13:19:14.441240072,worker_1
1,2024-05-13 13:19:19.155570984,2024-05-13 13:19:19.399858952,worker_1
2,2024-05-13 13:19:20.105675936,2024-05-13 13:19:20.320869923,worker_1
3,2024-05-13 13:19:21.000042915,2024-05-13 13:19:21.210474968,worker_1
4,2024-05-13 13:19:22.045061827,2024-05-13 13:19:22.237295151,worker_1
...,...,...,...
1000,2024-05-13 13:40:20.548667908,2024-05-13 13:40:21.422496080,server
1001,2024-05-13 13:40:25.284724951,2024-05-13 13:40:26.315452814,server
1002,2024-05-13 13:40:26.327105999,2024-05-13 13:40:27.170197964,server
1003,2024-05-13 13:40:27.180989981,2024-05-13 13:40:28.026266813,server


In [18]:
timeline = px.timeline(
    clients_and_server_df,
    x_start="start_time",
    x_end="end_time",
    color="name",
    title="Client logs",
    opacity=0.4,
    template="plotly_white"
)
# add vertical line to show when server logs started
for i in server_df["start_time"]:
    timeline.add_vline(x=i, line_dash="dash", line_color="red")
timeline.show()